In [1]:
import numpy as np
from time import ctime
import pickle
from ortools.sat.python import cp_model

Notebook for assignment 3 - graph colouring. 

The idea is to colour the network graph different colours such that no node has the same colour as its neighbours. We want to minimise the total colours used.

First uses a greedy approach (mostly pinched from some website), then a Constraint Programming approach.

Greedy approach is enough to get feasible but low quality solutions for all problems very quickly, and alright solutions on a few.

The CP approach (using Google's OR tools library) works quite well now, after leveraging the following tricks:

 - Symmetry breaking:
     - Any solution is symmetric in the values of colour assigned to the nodes. For example, an assignment 2 0 1 1 is the same as an 1 2 0 0, since while the colours have changed around, their relation to each other (and hence their feasibility) has not.
     - We can exploit this fact by imposing a condition that each colour can only be used if the next lower indexed colour has already been used. This means that for colour 1 can only be used if colour 0 has, colour 2 only if colour 1 has, etc. This constraint will be used in the search process, cutting out lots of symmetrical solutions.
 - Variable search ordering:
     - In CP, it usually makes sense to try and assign the 'hardest' variables first. This way, you don't waste time working on easy parts of the solution just to find they make the harder parts infeasible, meaning you have to start again. 
     - Here, the hardest parts of the solution are the nodes with the highest order (number of neighbouring nodes). To take it even further, for every node I calculated the total second order neighbours it had (neighbours of its neighbours) and searched according to this ordering from highest to lowest. 
 - Leveraging the greedy solution in different ways:
     - Firstly, setting a constraint that the objective value must be less than or equal to the greedy solutions objective. This helps speed up the search by immediatly pruning solutions with a higher objective value.
     - Secondly, it can help reduce the colour domain from [0, number of nodes-1] to [0, greedy_objective_function-1], since we have constrained the solution to have a maximum number of colours equal to the greedy objective value.
 - Multiprocessing and randomisation for search:
     - The search has been somewhat parellised using inbuilt ortools functionality.
     - The search has been set to use some randomisation in the values chosen, proved to be useful empirically.

## Data functions

In [2]:
def load_input_data(input_data):
    """
    Return input data as numpy array of [edge 1, edge 2]
    """

    # parse the input
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    num_nodes = int(firstLine[0])
    num_edges = int(firstLine[1])

    out_array = np.zeros((num_edges, 2))

    for i in range(num_edges):
        line = lines[i + 1]
        parts = line.split()
        out_array[i] = np.array([int(parts[0]), int(parts[1])])

    return out_array, num_nodes

In [3]:
input_file = open('data/gc_20_1','r').read()

In [4]:
edge_array, num_nodes = load_input_data(input_file)
edge_array

array([[0., 1.],
       [1., 2.],
       [1., 3.]])

In [5]:
def order_nodes_by_order_desc(edge_array):
    
    node_order_dict = {}
    
    for node_index in range(int(edge_array.max())+1):
        count_node_edges = (edge_array == node_index).sum()
        node_order_dict[node_index] = count_node_edges
        
    node_by_order_desc = [tup[0] for tup in sorted(node_order_dict.items(), key=lambda x: x[1], reverse=True)]
        
    return node_by_order_desc

In [6]:
d = order_nodes_by_order_desc(edge_array)
d

[1, 0, 2, 3]

In [7]:
def order_nodes_by_neighbours_order_descending(edge_array):
    
    node_neighbour_dict = {}
    
    for edge_index in range(edge_array.shape[0]):
        v0, v1 = int(edge_array[edge_index, 0]),  int(edge_array[edge_index, 1])
        if v0 not in node_neighbour_dict:
            node_neighbour_dict[v0] = []
        node_neighbour_dict[v0].append(v1)
        if v1 not in node_neighbour_dict:
            node_neighbour_dict[v1] = []
        node_neighbour_dict[v1].append(v0)
        
    node_neighbours_order_dict = {}
    for node, neighbours in node_neighbour_dict.items():
        node_neighbours_neighbors = 0
        for neighbour in neighbours:
            node_neighbours_neighbors += len(node_neighbour_dict[neighbour])
        node_neighbours_order_dict[node] = node_neighbours_neighbors
        
    out_list = [tup[0] for tup in sorted(node_neighbours_order_dict.items(), key=lambda x: x[1], reverse=True)]
        
    return out_list

In [8]:
order_nodes_by_neighbours_order_descending(edge_array)

[0, 1, 2, 3]

In [9]:
def prepare_output_data(solution_dict, is_provably_optimal=False):
    """
    Return output in specified format.
    """

    if is_provably_optimal:
        optimal = str(1)
    else:
        optimal = str(0)

    output_data = str(solution_dict['num_colours']) + ' ' + optimal + '\n'
    output_data += ' '.join(map(str, solution_dict['solution_array']))

    return output_data

In [16]:
prepare_output_data(get_solution_dict(nc_var, solv, num_nodes))

'2 0\n1 0 1 1'

## Greedy colouring algorithm

In [10]:
def greedy_colouring(edge_array):
    
    def color_nodes(graph):
        color_map = {}
        # Consider nodes in descending degree 
        for node in sorted(graph, key=lambda x: len(graph[x]), reverse=True):
            neighbor_colors = set(color_map.get(neigh) for neigh in graph[node])
            color_map[node] = next( 
                color for color in range(len(graph)) if color not in neighbor_colors
            )
        return color_map

    node_neighbour_dict = {}
    
    for edge_index in range(edge_array.shape[0]):
        v0, v1 = int(edge_array[edge_index, 0]),  int(edge_array[edge_index, 1])
        if v0 not in node_neighbour_dict:
            node_neighbour_dict[v0] = []
        node_neighbour_dict[v0].append(v1)
        if v1 not in node_neighbour_dict:
            node_neighbour_dict[v1] = []
        node_neighbour_dict[v1].append(v0)
        
    solution_dict = color_nodes(node_neighbour_dict)
        
    solution_array = np.zeros(int(edge_array.max()+1))
    obj_val = 0
    for i in range(int(edge_array.max()+1)):
        colour = solution_dict[i]
        solution_array[i] = colour
        obj_val = max(obj_val, colour)
        
    out_dict = {
        'solution_array': solution_array.astype(int),
        'num_colours': obj_val + 1
    }
        
    return out_dict


In [54]:
greedy_colouring(edge_array)

{'solution_array': array([14,  9,  9, 11,  8, 18,  1,  5,  0,  5, 16, 12,  1, 14,  7,  0, 14,
        13, 20,  7, 17, 12, 10,  6,  2, 11,  4, 13,  2,  4, 14, 15, 12,  5,
         4, 17, 17,  0,  3, 11, 13,  2, 11, 17,  2,  1,  7, 12, 15,  8,  8,
        15,  0, 13, 18,  3, 16, 10,  1, 19,  9,  7,  2,  6,  3,  9, 10, 16,
         6,  4]), 'num_colours': 21}

## CP model

In [11]:
def create_node_colour_variables(model, edge_array, num_nodes, num_colours=None):
    
    # fine for now
    if num_colours is None:
        num_colours = num_nodes 
    
    node_colour_variables = {}
    for counter, node_index in enumerate(order_nodes_by_neighbours_order_descending(edge_array)):
        node_colour_variables[node_index] = model.NewIntVar(0, num_colours-1, 'node_%i' % counter)
    
    return node_colour_variables, num_colours

def create_node_colour_constraints(model, edge_array, node_colour_variables):
    
    for edge_index in range(edge_array.shape[0]):
        model.Add(node_colour_variables[edge_array[edge_index, 0]] != node_colour_variables[edge_array[edge_index, 1]])
        
    return model

def create_node_colour_bool_variables(model, edge_array, num_nodes, num_colours=None):
    
    # fine for now
    if num_colours is None:
        num_colours = num_nodes 
    
    node_colour_bool_variables = {}
    for counter, node_index in enumerate(order_nodes_by_neighbours_order_descending(edge_array)):
        for colour_index in range(num_colours):
            node_colour_bool_variables[(node_index, colour_index)] = \
                model.NewBoolVar('node_%i_colour_%i' % (counter, colour_index))
    
    return node_colour_bool_variables

def create_node_colour_bool_constraints(model, node_colour_bool_variables, 
                                        node_colour_variables, num_nodes, num_colours):
    
    for node_index in range(num_nodes):
        for colour_index in range(num_colours):
            model.Add(node_colour_variables[node_index] == colour_index).\
                OnlyEnforceIf(node_colour_bool_variables[(node_index, colour_index)])
            model.Add(node_colour_variables[node_index] != colour_index).\
                OnlyEnforceIf(node_colour_bool_variables[(node_index, colour_index)].Not())
            
    return model

def create_node_colour_bool_constraints(model, node_colour_bool_variables, 
                                        node_colour_variables, num_nodes, num_colours):
    
    for node_index in range(num_nodes):
        for colour_index in range(num_colours):
            model.Add(node_colour_variables[node_index] == colour_index).\
                OnlyEnforceIf(node_colour_bool_variables[(node_index, colour_index)])
            model.Add(node_colour_variables[node_index] != colour_index).\
                OnlyEnforceIf(node_colour_bool_variables[(node_index, colour_index)].Not())
            
    return model

def create_colour_used_variables(model, num_colours):
    
    colour_used_variables = {}
    for colour_index in range(num_colours):
        colour_used_variables[colour_index] = model.NewBoolVar('colour_%i' % colour_index)
        
    return colour_used_variables

def create_colour_used_constraints(model, node_colour_bool_variables, colour_used_variables, num_nodes, num_colours):
    
    for colour_index in range(num_colours):
        model.AddMaxEquality(colour_used_variables[colour_index],[node_colour_bool_variables[(node_index, colour_index)]
                                                                  for node_index in range(num_nodes)])
        
    return model

def create_colour_symmetry_breaking_constraints(model, colour_used_variables, num_colours):
    
    for colour_index in range(num_colours-1):
        model.Add(colour_used_variables[colour_index] >= colour_used_variables[colour_index+1])
        
    return model

def create_objective(model, colour_used_variables, num_colours):
    
    # create objective value variable
    obj_val_var = model.NewIntVar(0, num_colours-1, 'num_distinct_colours')
    
    # add constraint to set the variable
    model.Add(obj_val_var == sum([colour_used_variables[colour_index] for colour_index in range(num_colours)]))
    
    model.Minimize(obj_val_var)
    
    return model, obj_val_var
              
    
def create_model(edge_array, num_nodes, max_obj=None):
    
    # create model
    model = cp_model.CpModel()
    
    # create variables
    node_colour_variables, num_colours = create_node_colour_variables(model, edge_array, num_nodes, num_colours=max_obj)
    node_colour_bool_variables = create_node_colour_bool_variables(model, edge_array, num_nodes, num_colours=max_obj)
    colour_used_variables = create_colour_used_variables(model, num_colours)
    
    
    # add constraints
    model = create_node_colour_bool_constraints(model, node_colour_bool_variables, node_colour_variables, 
                                                num_nodes, num_colours)
    model = create_node_colour_constraints(model, edge_array, node_colour_variables)
    model = create_colour_used_constraints(model, node_colour_bool_variables, colour_used_variables, num_nodes, num_colours)
    model = create_colour_symmetry_breaking_constraints(model, colour_used_variables, num_colours)
    
    
    # add objective
    model, obj_val_variable = create_objective(model, colour_used_variables, num_colours)
    
    # if we have a known max value for the objective, restrict to be lower than this
    if max_obj is not None:
        model.Add(obj_val_variable <= max_obj)
    
    model.AddDecisionStrategy([node_colour_variables[node_index] 
                               for node_index in order_nodes_by_neighbours_order_descending(edge_array)], 
                               cp_model.CHOOSE_FIRST , cp_model.SELECT_MIN_VALUE)
    
    return model, node_colour_variables, node_colour_bool_variables , colour_used_variables , obj_val_variable

def solve_model(model, max_solve_time=10):
    
    # solve model
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = max_solve_time
    solver.parameters.search_branching = cp_model.FIXED_SEARCH
    solver.parameters.num_search_workers = 16
    solver.parameters.randomize_search = True
    status = solver.Solve(model)
    
    return model, solver, status

def print_results(model, variables, obj_val_variable, solver):
    
    max_obj = 0
    
    for node_index, node_var in variables.items():
        val = solver.Value(node_var)
        max_obj = max(max_obj, val)
        print('Node ' + str(node_index) + ': ' + str(val))
        
    print('Objective: ' + str(solver.Value(obj_val_variable)))

In [12]:
mod, nc_var, ncb_var, cu_var, obj_val_var = create_model(edge_array, num_nodes, max_obj=9)

In [13]:
print(ctime())
mod, solv, stat = solve_model(mod, max_solve_time=300)
print(ctime())

Mon Feb  3 14:34:14 2020
Mon Feb  3 14:34:14 2020


In [52]:
%timeit solve_model(mod, max_solve_time=30)

150 ms ± 8.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
print_results(mod, nc_var, obj_val_var, solv)

Node 45: 0
Node 35: 1
Node 47: 1
Node 33: 2
Node 7: 0
Node 32: 4
Node 44: 4
Node 27: 5
Node 29: 3
Node 6: 4
Node 16: 0
Node 11: 0
Node 15: 1
Node 20: 3
Node 42: 0
Node 43: 0
Node 19: 2
Node 1: 4
Node 9: 5
Node 31: 5
Node 22: 2
Node 28: 5
Node 8: 2
Node 12: 1
Node 26: 1
Node 41: 3
Node 17: 5
Node 3: 3
Node 4: 2
Node 49: 3
Node 46: 3
Node 5: 1
Node 34: 3
Node 36: 2
Node 40: 4
Node 37: 2
Node 24: 1
Node 48: 3
Node 18: 3
Node 2: 4
Node 23: 1
Node 14: 4
Node 30: 4
Node 10: 1
Node 25: 2
Node 13: 0
Node 39: 3
Node 38: 4
Node 21: 5
Node 0: 5
Objective: 6


In [14]:
def get_solution_dict(variables, solver, num_nodes):
    
    max_obj = 0
    
    solution_array = np.zeros(num_nodes)
    
    for node_index, node_var in variables.items():
        val = solver.Value(node_var)
        solution_array[node_index] = val
        max_obj = max(max_obj, val)

    solution_dict = {
        'solution_array': solution_array.astype(int),
        'num_colours': max_obj+1
    }
        
    return solution_dict

In [15]:
sol_d = get_solution_dict(nc_var, solv, num_nodes)
sol_d

{'solution_array': array([1, 0, 1, 1]), 'num_colours': 2}

In [71]:
with open('Assignment_3_Question_2.pickle', 'wb') as handle:
    pickle.dump(sol_d, handle)

In [197]:
def solve_it_cp(edge_array, num_nodes, max_solve_time=60):
    
    greedy_solution_dict = greedy_colouring(edge_array)
    
    max_obj = greedy_solution_dict['num_colours']

    model, nc_vars, ncb_vars, cu_vars, obj_val_var = create_model(edge_array, num_nodes, max_obj=max_obj)

    model, solv, stat = solve_model(model, max_solve_time=max_solve_time)

    solution_dict = get_solution_dict(nc_vars, solv, num_nodes)

    return solution_dict

In [4]:
with open('Assignment_3_Question_2.pickle','rb') as handle:
    solution_dict = pickle.load(handle)
solution_dict

{'solution_array': array([11,  5,  5,  6,  0,  7, 12, 15,  2, 15,  2,  9,  0, 16, 16,  8, 16,
         4,  3, 14, 10,  9,  8,  8,  7,  0,  4,  6, 13,  1,  3,  3,  9, 10,
         7, 12,  8, 11,  4, 13,  6, 14,  1, 10, 14,  3, 16,  9, 13,  0, 11,
        13,  2,  6, 12,  4,  2,  7, 16, 13,  5, 12, 14,  1,  5,  5, 15, 15,
         1,  7]), 'num_colours': 17}

In [6]:
prepare_output_data(solution_dict)

'17 0\n11 5 5 6 0 7 12 15 2 15 2 9 0 16 16 8 16 4 3 14 10 9 8 8 7 0 4 6 13 1 3 3 9 10 7 12 8 11 4 13 6 14 1 10 14 3 16 9 13 0 11 13 2 6 12 4 2 7 16 13 5 12 14 1 5 5 15 15 1 7'